In [1]:
import pandas as pd
import numpy as np
from numba import njit


In [2]:
df = pd.util.testing.makeTimeDataFrame()

/workspace/.pyenv_mirror/user/current/lib/python3.8/site-packages/pandas/util/__init__.py:16: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing


In [3]:
df

,A,B,C,D
2000-01-03,0.659678,0.869418,1.820727,-0.388127
2000-01-04,0.141383,-1.484501,-0.994924,-0.702117
2000-01-05,0.875512,-0.491880,0.860127,0.166339
2000-01-06,1.037864,0.418483,-0.379245,-0.542846
2000-01-07,0.613910,0.397445,-0.519262,-0.038640
2000-01-10,-1.217058,1.665590,1.750582,-0.497171
2000-01-11,-0.202558,-0.175702,0.233559,1.714532
2000-01-12,-0.349769,-0.002965,0.112607,-1.401419
2000-01-13,-0.866519,-1.069092,1.014726,0.556989
2000-01-14,0.759525,0.908011,-0.079572,1.213686


In [10]:
@njit
def calculate_go_with_flow(array):
    #最初开始 rolling 时只有一行数据，不能计算相关系数，因此需要加入容错机制
    if array.shape[0] >= 2:
        #计算各股票与其他股票的“高低额差”的 Pearson 相关系数
        correlation =np.corrcoef(array,rowvar=False)
        #将相关系数取绝对值
        correlation =np.abs(correlation)
        # 将对角线元素设为空值
        np.fill_diagonal(correlation,np.NaN)
        #计算每只股票与其余股票相关系数绝对值的均值
        #由于 numba 不支持在 np.mean() 中设定参数，因此我们只能手动构造数组，以实现对每行求均值
        correlation_mean =[]
        stock_num =array.shape[1]
        for i in range(stock_num):
            # 计算第 i 只股票与其他股票的相关系数的绝对值的均值
            correlation_mean.append(np.nanmean(correlation[i,:]))
    #由于 numba 不支持 .to list() 方法，因此我们需要手动构造列表，返回多个值
    result = np.zeros(stock_num)
    for i in range(stock_num):
        result[i] = correlation_mean[i]
    return result

In [13]:
df.rolling(20, method="table").apply(
calculate_go_with_flow,raw=True, engine="numba"
)

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
Failed in nopython mode pipeline (step: nopython frontend)
No implementation of function Function(<function corrcoef at 0x7f28141b8c10>) found for signature:
 
 >>> corrcoef(array(float64, 2d, A), rowvar=Literal[bool](False))
 
There are 2 candidate implementations:
  - Of which 1 did not match due to:
  Overload in function 'np_corrcoef': File: numba/np/arraymath.py: Line 3041.
    With argument(s): '(array(float64, 2d, A), rowvar=bool)':
   Rejected as the implementation raised a specific error:
     TypingError: Failed in nopython mode pipeline (step: nopython frontend)
   No implementation of function Function(<function cov at 0x7f28141b8a60>) found for signature:
    
    >>> cov(array(float64, 2d, A), none, bool)
    
   There are 2 candidate implementations:
         - Of which 2 did not match due to:
         Overload in function 'np_cov': File: numba/np/arraymath.py: Line 2982.
           With argument(s):